<a href="https://colab.research.google.com/github/sfatimazahra/MDiator/blob/master/speechtranslator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing dependancies

In [18]:
!pip install openai google-cloud-speech

In [19]:
!pip install openai flask flask_cors flask_ngrok2 pyngrok

In [20]:

from google.colab import userdata
import os
import pyngrok

# Securely load the API keys and tokens from Colab Secrets
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
pyngrok.conf.get_default().auth_token = userdata.get("NGROK_AUTH_TOKEN")
print("Environment set up successfully!")

Environment set up successfully!


API ENDPOINT


In [ ]:
import base64
from flask import Flask, request, jsonify
from flask_cors import CORS
from openai import OpenAI
import tempfile
import os
import time
from pyngrok import ngrok

# Initialize the OpenAI client
client = OpenAI()

app = Flask(__name__)
# Enable CORS to allow the frontend to call this server
CORS(app)

def transcribe_and_translate_audio(audio_bytes, target_language, model):
    """
    Transcribes and translates audio using OpenAI's APIs.
    """
    try:
        # Create a temporary file to save the audio data
        with tempfile.NamedTemporaryFile(suffix=".webm", delete=False) as temp_file:
            temp_file.write(audio_bytes)
            temp_file_path = temp_file.name

        # Transcribe the audio and automatically detect the source language
        transcription_response = client.audio.transcriptions.create(
            model="whisper-1",
            file=open(temp_file_path, "rb"),
            language=None  # Automatically detect language
        )

        transcript = transcription_response.text
        source_language = transcription_response.language

        # Check if Whisper successfully detected a language
        if not source_language:
            os.remove(temp_file_path)
            return None, None, "Could not detect source language."

        # Translate the text to the target language using a generative model.
        translation_prompt = f"You are a professional healthcare translator. Translate the following medical transcription from {source_language} to {target_language}. Ensure the translation is accurate and uses appropriate medical terminology."

        translation_response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": translation_prompt},
                {"role": "user", "content": transcript}
            ]
        )
        translated_text = translation_response.choices[0].message.content

        # Clean up the temporary file
        os.remove(temp_file_path)

        return transcript, translated_text, None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None, str(e)

@app.route("/process-audio", methods=["POST"])
def process_audio():
    # Receive the audio file, target language, and model from the frontend
    audio_file = request.files.get("audio")
    target_language = request.form.get("target_language")
    model = request.form.get("model")

    if not audio_file or not target_language or not model:
        return jsonify({"error": "Missing audio file, target language, or model."}), 400

    audio_bytes = audio_file.read()
    source_text, translated_text, error = transcribe_and_translate_audio(audio_bytes, target_language, model)

    if error:
        return jsonify({"error": error}), 500

    return jsonify({
        "source_transcript": source_text,
        "translated_transcript": translated_text
    })

if __name__ == "__main__":
    # Start ngrok tunnel and get the public URL
    public_url = ngrok.connect(5000)
    print(f"Ngrok URL: {public_url}")

    # Run the Flask app
    app.run()

Ngrok URL: NgrokTunnel: "https://1bf44fb44dd0.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [30/Aug/2025 11:59:23] "POST /process-audio HTTP/1.1" 500 -


An error occurred: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
